In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time

CHROMEDRIVER_PATH = "/Users/parakhchaudhary/chromedriver_135/chromedriver-mac-x64/chromedriver"

options = Options()
options.add_argument("--headless")  # Optional: remove to see browser

service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)


In [2]:

base_url = "https://www.shl.com/products/product-catalog/?start={}&type=1"
all_data = []


In [3]:
for offset in range(0, 50, 12):
    url = f"https://www.shl.com/products/product-catalog/?start={offset}&type=1"
    print(f"Visiting {url}")
    driver.get(url)
    time.sleep(2)  # Let JS render fully

    soup = BeautifulSoup(driver.page_source, "html.parser")
    tables = soup.find_all("table")
    if len(tables) < 2:
        continue

    table = tables[-1]  # Individual Test Solutions
    rows = table.find_all("tr")[1:]

    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 4:
            continue

        a_tag = cols[0].find("a")
        name = a_tag.text.strip()
        href = a_tag['href'].strip()
        remote = "Yes"
        adaptive = "Yes" if cols[2].find("svg") else "No"
        test_type = ''.join(span.text for span in cols[3].find_all("span"))

        all_data.append({
            "name": name,
            "url": href,
            "remote_testing": remote,
            "adaptive_support": adaptive,
            "test_type": test_type
        })


Visiting https://www.shl.com/products/product-catalog/?start=0&type=1
Visiting https://www.shl.com/products/product-catalog/?start=12&type=1
Visiting https://www.shl.com/products/product-catalog/?start=24&type=1
Visiting https://www.shl.com/products/product-catalog/?start=36&type=1
Visiting https://www.shl.com/products/product-catalog/?start=48&type=1


In [4]:

# Save final file
with open("shl_final_catalog2.csv", "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=all_data[0].keys())
    writer.writeheader()
    writer.writerows(all_data)

print("✅ Done. Collected", len(all_data), "entries")


✅ Done. Collected 12 entries
